In [1]:
import cv2 as cv
import matplotlib.pyplot as plt 
%matplotlib inline
plt.rcParams["figure.figsize"]=[5,5]

In [2]:
figuras=cv.imread("imagenes/texto.jpg")
gray=cv.cvtColor(figuras, cv.COLOR_RGB2GRAY)

In [3]:
def histograma(gray, hxb=False, hyb=False):
    histx=[]
    histy=[]
    if hyb:
        for row in gray:
            histy.append(sum(row))
    if hxb:
        for col in gray.T:
            histx.append(sum(col))
    return histx, histy

In [4]:
def cortes(hbin):
    v_ant=hbin[0]
    cortes_index=[]
    for index, h in enumerate(hbin):
        v_sig=h
        if v_sig!=v_ant:
            cortes_index.append(index)
        v_ant=v_sig
    return cortes_index, len(cortes_index)

In [29]:
def corte_val(corte, tam, th_corte, th_borde):
    corte_vals=[]
    tam_v=0
    for i, dat in enumerate(range(int(tam/2))):
        len_corte=corte[2*i+1]-corte[2*i]
        corte_vals.append([corte[2*i]-th_borde,corte[2*i+1]+th_borde]) if len_corte>th_corte else _
        tam_v+=1*(len_corte>th_corte)
    return corte_vals, tam_v

In [104]:
def extract_objects(gray_image, threshold=240, th_corte=10, th_border=5):
    figuras=[]
    NumFiguras=0
    threhold, gray_image_bool=cv.threshold(gray, threshold, 255, cv.THRESH_BINARY_INV)
    hx, hy=histograma(gray_image_bool, hxb=True, hyb=True)
    hxbin=[h>th_corte for h in hx]
    hybin=[h>th_corte for h in hy]

    cortex, tamx= cortes(hxbin)
    cortey, tamy= cortes(hybin)

    b_h=tamx>tamy
    corte_v, tam_v=corte_val(cortex, tamx, th_corte, th_border) if b_h else corte_val(cortey, tamy, th_corte, th_border)
    for i in range(tam_v):
        img_c=gray_image_bool[:,corte_v[i][0]:corte_v[i][1]] if b_h else gray_image_bool.T[:,corte_v[i][0]:corte_v[i][1]]
        hx, hy=histograma(img_c, hxb=False, hyb=True)
        hybin=[h>30 for h in hy]
        cortey, tamy= cortes(hybin)
        corte, tam= corte_val(cortey, tamy, th_corte, th_border)
        NumFiguras+=tam
        for j in range(tam):
            figuras.append(img_c[corte[j][0]:corte[j][1],:].T)
    return figuras, NumFiguras

In [178]:
def clean_letters(figuras):
    Figuras=[]
    for fig in range(len(figuras)):
        hx, hy=histograma(figuras[fig][:], hxb=False, hyb=True)
        hybin=[h>0 for h in hy]
        cortey, tamy= cortes(hybin)
        Figuras.append(figuras[fig][cortey[tamy-2]-1:cortey[tamy-1]+1,:])
    return Figuras

In [179]:
figuras, NumFiguras=extract_objects(gray, threshold=240, th_corte=1, th_border=1)


In [180]:
figuras_clean=clean_letters(figuras)

In [53]:
from ipywidgets import interact, IntSlider

In [181]:
@interact(numero=IntSlider(6,min=1,max=30))
def mostrar_imagenes(numero):
    plt.imshow(figuras_clean[numero-1][:])

interactive(children=(IntSlider(value=6, description='numero', max=30, min=1), Output()), _dom_classes=('widge…

In [186]:
for i in range(NumFiguras):
    cv.imwrite("TextoExtraido/signo{}.jpg".format(i+1),figuras_clean[i][:])

True